# Ejemplo búsquedas con R-Tree

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 08 de junio de 2025

Para ejecutar el código en este notebook debe instalar dos librerías:
```
pip install rtree matplotlib
```
Ver documentación de la librería `rtree` en: https://rtree.readthedocs.io/en/latest/class.html

Primero se realizará un ejemplo con datos de dos dimensiones. Luego se realizará un ejemplo con vectores 128 dimensiones.

# Ejemplo 1: R-tree con datos 2D

In [ ]:
import numpy

dataset_r = numpy.load("dataset_2d_r.npy")
dataset_q = numpy.load("dataset_2d_q.npy")

print("Dataset Q={} R={}".format(dataset_q.shape, dataset_r.shape))

In [ ]:
import matplotlib.pyplot as plt

def dibujar_dataset(datar, dataq):
    x = datar[:,0]
    y = datar[:,1]
    plt.scatter(x, y, label="dataset_r")
    x2 = dataq[:,0]
    y2 = dataq[:,1]
    plt.scatter(x2, y2, label="dataset_q")
    plt.legend(bbox_to_anchor=(1,1))
    plt.show()
    
dibujar_dataset(dataset_r, dataset_q)

## Crear índice R-tree

In [ ]:
import time 
import datetime
from rtree import index

def crear_indice(descriptores):
    print("creando indice para r={}".format(descriptores.shape))

    p = index.Property()
    p.dimension = descriptores.shape[1]

    idx = index.Index(properties=p, interleaved=True)

    descriptores_boxes = numpy.hstack((descriptores, descriptores))

    for row in range(descriptores_boxes.shape[0]):
        if (row+1) % 5000 == 0:
            print("[{}]   {} / {}...".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), row+1, descriptores.shape[0]))
        idx.insert(id=row, coordinates=descriptores_boxes[row])
        
    return idx

t0 = time.time()
r_tree = crear_indice(dataset_r)
t1 = time.time()
print("tiempo creacion indice: {:.1f} segs".format(t1-t0))

## Búsquedas del NN usando el R-tree

In [ ]:
import datetime

def busqueda_nn(idx, descriptores):
    print("busqueda del mas cercano q={}".format(descriptores.shape))

    descriptores_boxes = numpy.hstack((descriptores, descriptores))

    all_nns = []
    for row in range(descriptores_boxes.shape[0]):
        if (row+1) % 100 == 0:
            print("[{}]   {} / {}...".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), row+1, descriptores.shape[0]))
        nns = idx.nearest(coordinates=descriptores_boxes[row], num_results=1, objects=False)
        nn = next(nns)
        all_nns.append(nn)
    
    return all_nns

t0 = time.time()
nns = busqueda_nn(r_tree, dataset_q)
t1 = time.time()
print("tiempo busqueda NN: {:.1f} segs".format(t1-t0))

## Resultado de la búsqueda

In [ ]:
for row in range(dataset_q.shape[0]):
    print("{} -> {}".format(dataset_q[row], dataset_r[nns[row]]))

El R-tree funciona bien con datos espaciales (2D o 3D).  
A continuación se prueba el R-tree con datos de mayor dimensión.

# Ejemplo 2: R-tree con dataset de alta dimensionalidad

Se usa el mismo dataset para el ejemplo de kdtree (**Q** son 5 mil vectores y **R** son 100 mil vectores de dimensión 128).

El R-tree funciona bien con datos espaciales (2D o 3D), sin embargo con datos de alta dimensión no tiene buena performance debido a que los MBRs tienden a aumentar mucho. Como toman el mínimo y máximo de cada dimensión, los rectángulos contienen mucho volumen.


In [ ]:
import numpy
import time

datasetA_q = numpy.load("dataset_a_q.npy")
datasetA_r = numpy.load("dataset_a_r.npy")

print("Dataset A: conjunto_Q={} conjunto_R={}".format(datasetA_q.shape, datasetA_r.shape))

## Crear R-tree (toma unos 5 minutos)

In [ ]:
t0 = time.time()
rtree_A = crear_indice(datasetA_r)
t1 = time.time()
print("tiempo creacion indice: {:.1f} segs ({:.1f} mins)".format(t1-t0, (t1-t0)/60))

## Buscar en el R-tree (toma de 10 a 15 minutos)

In [ ]:
t0 = time.time()
nns_A = busqueda_nn(rtree_A, datasetA_q)
t1 = time.time()
print("tiempo busqueda NN: {:.1f} segs ({:.1f} mins)".format(t1-t0, (t1-t0)/60))

## Comparar con linear scan del anexo anterior

In [ ]:
# cargar resultado del linear scan que fue guardado por el Anexo anterior
nns_ideal = numpy.load('dataset_a-nns-linear_scan.npy')

def evaluar_busqueda(nns):
    correctas = 0
    incorrectas = 0
    for i in range(len(nns_ideal)):
        if nns[i] == nns_ideal[i]: 
            correctas += 1
        else:
            incorrectas += 1 
    precision = correctas / (correctas + incorrectas)
    print("  correctas={} incorrectas={} precision={:.1%}".format(correctas, incorrectas, precision))

evaluar_busqueda(nns_A)


Los resultados son idénticos en 4.881 de las 5.000 consultas. La diferencia en la respuesta de 119 consultas que se podría deber a que al calcular la distancia euclidiana entre floats, no obtuvo exactamente el mismo valor, por lo que encontró otro elemento que era el más cercano.

En el Anexo anterior se ve que el Linear Scan demora cerca de 30 segundos en comparar todos los elementos. En este caso resolver la búsqueda exacta con R-tree demora más de 10 minutos (!!) por lo que no se ve como buena opción para reemplazar el linear scan.

# Pregunta

Notar la diferencia en performance entre el R-Tree y el k-d tree cuando se usa un dataset de vectores de 128-d. En el anexo anterior se vio que el k-d tree para búsqueda exacta funciona levemente mejor que el Linear scan, mientras que en este ejemplo se ve que el R-tree es mucho más lento que el Linear Scan.

**¿A que se puede deber que el R-Tree funcione tan mal en este dataset (mucho peor que linear scan y que k-d tree)? ¿Por qué el k-d tree no tiene ese efecto?** 